In [ ]:
import numpy as np
from PIL import Image
import requests
from io import BytesIO

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transform

In [ ]:
transform = transform.Compose([
    transform.ToTensor(),
    transform.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [ ]:
train_data = torchvision.datasets.CIFAR10(root='./data',train=True,transform = transform,download=True)
test_data = torchvision.datasets.CIFAR10(root='./data',train=False,transform = transform,download=True)

train_dataloader = torch.utils.data.DataLoader(train_data,batch_size = 32,shuffle=True,num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_data,batch_size = 32,shuffle=True,num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
image,label = train_data[0]
image.size()

torch.Size([3, 32, 32])

In [ ]:
class_names = ['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [ ]:
class NeuralNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(3,12,5)  # next --> ((32-5)/stride)+1=28. 12,28,28.  kernel-5*5,produces 12 featuremaps
    self.pool = nn.MaxPool2d(2,2)   # --> 12,14,14
    self.conv2 = nn.Conv2d(12,24,5) #-->  1+(14-5)/1 (24,10,10) -->  (24,5,5) ->flatten-->(24*5*5)

    self.fc1 = nn.Linear(24*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [ ]:
model = NeuralNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum=0.9)

In [ ]:
for epoch in range(50):
    print(f'Training epoch: {epoch}...')
    running_loss = 0.0

    for i, data in enumerate(train_dataloader):
        inputs, label = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, label)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_dataloader):.4f}')

Training epoch: 0...
Loss: 2.1728
Training epoch: 1...
Loss: 1.7483
Training epoch: 2...
Loss: 1.5282
Training epoch: 3...
Loss: 1.4164
Training epoch: 4...
Loss: 1.3311
Training epoch: 5...
Loss: 1.2547
Training epoch: 6...
Loss: 1.1908
Training epoch: 7...
Loss: 1.1332
Training epoch: 8...
Loss: 1.0796
Training epoch: 9...
Loss: 1.0367
Training epoch: 10...
Loss: 0.9922
Training epoch: 11...
Loss: 0.9496
Training epoch: 12...
Loss: 0.9153
Training epoch: 13...
Loss: 0.8766
Training epoch: 14...
Loss: 0.8438
Training epoch: 15...
Loss: 0.8120
Training epoch: 16...
Loss: 0.7792
Training epoch: 17...
Loss: 0.7461
Training epoch: 18...
Loss: 0.7213
Training epoch: 19...
Loss: 0.6937
Training epoch: 20...
Loss: 0.6707
Training epoch: 21...
Loss: 0.6442
Training epoch: 22...
Loss: 0.6202
Training epoch: 23...
Loss: 0.5991
Training epoch: 24...
Loss: 0.5773
Training epoch: 25...
Loss: 0.5528
Training epoch: 26...
Loss: 0.5345
Training epoch: 27...
Loss: 0.5139
Training epoch: 28...
Loss: 0.

In [ ]:
torch.save(model.state_dict(),'train_net1.pth')

In [ ]:
model = NeuralNet()
model.load_state_dict(torch.load('train_net1.pth'))

<ipython-input-46-8ebe4e15a628>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('train_net1.pth'))


<All keys matched successfully>

In [ ]:
correct = 0
total = 0

model.eval()

with torch.no_grad():
    for data in test_dataloader:
        images, labels = data  # Ensure you unpack data correctly
        outputs = model(images)  # Use images instead of inputs
        _, pred = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

accuracy = (correct * 100) / total
print(f'The accuracy is: {accuracy}%')


The accuracy is: 65.9%


In [ ]:
new_transform = transform.Compose([
    transform.Resize((32, 32)),  # Resize to 32x32
    transform.ToTensor(),  # Convert to tensor
    transform.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

# Function to load and transform the image
def load_img(img_path):
    response = requests.get(img_path)
    image = Image.open(BytesIO(response.content))  # Open image from byte data
    image = new_transform(image)  # Apply transformations
    image = image.unsqueeze(0)  # Add batch dimension
    return image

# Image URLs
image_paths = [
    'https://www.nylabone.com/-/media/project/oneweb/nylabone/images/dog101/10-intelligent-dog-breeds/golden-retriever-tongue-out.jpg',
    'https://media.gq.com/photos/6508829d305ef4e0229049b3/master/w_2240,c_limit/plane.jpg',
    'https://encrypted-tbn1.gstatic.com/images?q=tbn:ANd9GcQjr-6m4fJFTmH4sIqOHIMJbQBTMw4JhG-CcVscjg5N-fDXF6ku',
    'https://cdn.speedsize.com/eb8d0010-7300-4129-8a6d-74bc221f9caf/https://www.virginvoyages.com/dam/jcr:820ffd9b-f003-4e99-84f1-cc3e592b9da3/scarlet%20lady-2252x1266.png',
    'https://woofwell.com/cdn/shop/files/Golden-Retriever-Health-WoofWell-Breed-Specific-Dog-Supplements_1600x.jpg?v=1621360789',
    'https://www.nylabone.com/-/media/project/oneweb/nylabone/images/dog101/top-10-lists/10-intelligent-dog-breeds.jpg?h=318&iar=0&w=720&hash=BDE1A53E84C77A8C1C4DA40F79DE0915'
]

# Load images
images = [load_img(img) for img in image_paths]

# Model evaluation
model.eval()
with torch.no_grad():
    for image in images:
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        print(f'The prediction: {class_names[predicted.item()]}')

The prediction: dog
The prediction: plane
The prediction: cat
The prediction: ship
The prediction: cat
The prediction: plane
